In [ ]:
# 1. Installa Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Avvia il server di Ollama in background
!nohup ollama serve &

# 3. Dagli 5 secondi per avviarsi
import time
time.sleep(5)

print("Ollama installato e avviato.")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
Ollama installato e avviato.


In [ ]:
!ollama pull qwen3-vl:8b

In [ ]:
!pip install ollama

In [ ]:
!ollama list

NAME           ID              SIZE      MODIFIED      
qwen3-vl:8b    901cae732162    6.1 GB    9 seconds ago    


In [ ]:
import zipfile
import os

zip_path = "cm1_test_dataset.zip"
extract_path = "cm1-cover"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Immagini estratte in: {extract_path}")
else:
    print("Nessun file zip trovato, procedo assumendo che la cartella esista già.")

Immagini estratte in: cm1-cover


In [ ]:
import ollama
import json
import os
import base64
import zipfile
from tqdm import tqdm

# --- CONFIGURAZIONE ---
ZIP_FILE_PATH = "cm1_test_dataset.zip" # Il file caricato
OUTPUT_FILE = "predictions_qwen3_8b_50samples.json"
MODEL_NAME = "qwen3-vl:8b"

# Prompt
PROMPT_TEXT = """
Estrarre da questo documento in formato JSON i campi: Name, Vorname, Geb-Dat.
Regole importanti:
1. Scrivi 'Name' e 'Vorname' tutto in MAIUSCOLO.
2. Converti 'Geb-Dat' nel formato standard YYYY-MM-DD (es. 1940-12-31).
3. Se un campo è vuoto, usa una stringa vuota.
"""

all_predictions = {}

def get_clean_json(response_text):
    if "```json" in response_text:
        return response_text.split("```json")[-1].split("```")[0].strip()
    elif "```" in response_text:
        return response_text.split("```")[-1].split("```")[0].strip()
    return response_text.strip()

if not os.path.exists(ZIP_FILE_PATH):
    print(f"ERRORE: Carica {ZIP_FILE_PATH} su Colab!")
else:
    print(f"Apro lo zip e inizio l'elaborazione...")
    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as myzip:
        # Filtra i file immagine (ignora cartelle vuote o file macosx)
        file_list = [f for f in myzip.namelist() if f.lower().endswith(('.jpg', '.jpeg', '.png')) and "__MACOSX" not in f]

        for file_path in tqdm(file_list):

            # L'ID della GT è il nome della cartella padre
            gt_id = os.path.dirname(file_path)

            if not gt_id: continue

            try:
                # Leggi bytes
                with myzip.open(file_path) as f:
                    img_bytes = f.read()
                    img_base64 = base64.b64encode(img_bytes).decode('utf-8')

                # Inferenza
                response = ollama.chat(
                    model=MODEL_NAME,
                    messages=[{
                        'role': 'user',
                        'content': PROMPT_TEXT,
                        'images': [img_base64]
                    }],
                    stream=False
                )

                # Parsing
                json_txt = get_clean_json(response['message']['content'])
                data = json.loads(json_txt)

                # Normalizza a lista
                if isinstance(data, dict):
                    # Gestione wrapper "entries" o dizionario singolo
                    if len(data) == 1 and isinstance(list(data.values())[0], list):
                         final_data = list(data.values())[0]
                    else:
                         final_data = [data]
                elif isinstance(data, list):
                    final_data = data
                else:
                    final_data = []

                # Salva con l'ID corretto (quello della cartella)
                all_predictions[gt_id] = final_data

            except Exception as e:
                print(f"Errore su {gt_id}: {e}")
                all_predictions[gt_id] = []

            # Salvataggio frequente
            if len(all_predictions) % 5 == 0:
                with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
                    json.dump(all_predictions, f, indent=2, ensure_ascii=False)

    # Finale
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(all_predictions, f, indent=2, ensure_ascii=False)
    print("Finito! Scarica il file JSON.")

Apro lo zip e inizio l'elaborazione...


 22%|██▏       | 11/50 [21:19<1:41:42, 156.48s/it]

Errore su 79655593: Expecting value: line 1 column 1 (char 0)


 24%|██▍       | 12/50 [24:50<1:49:29, 172.88s/it]

Errore su 78933036: Expecting value: line 1 column 1 (char 0)


 64%|██████▍   | 32/50 [59:10<31:26, 104.79s/it]

Errore su 79805118: Expecting value: line 1 column 1 (char 0)


100%|██████████| 50/50 [1:34:39<00:00, 113.59s/it]

Finito! Scarica il file JSON.
